### Set Seed Cố Định

In [2]:
import os
import torch
import random
import numpy as np

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
print("Seed set to 42")

Seed set to 42


## Nạp Mô Hình và Tài Nguyên

In [6]:
import sys
import time
import json
import glob
import cv2
import pandas as pd

sys.path.append('/code')
from predict import AeroEyesModel
INPUT_FOLDER = "/data/samples/"
OUTPUT_JSON = "/result/jupyter_submission.json"
OUTPUT_CSV = "/result/time_submission.csv"

model = AeroEyesModel()
print("Model loaded for inference.")

🖥️ Running on: cpu
⏳ Loading YOLO-World...


FileNotFoundError: Model not found at /saved_models/models.pt

## Đọc Nội Dung Các Test Case

In [13]:
import glob

DATA_ROOT = r"data\samples"
test_cases = glob.glob(os.path.join(DATA_ROOT, "*", "drone_video.mp4"))
print(f"Found {len(test_cases)} test videos.")

Found 6 test videos.


## Thực Hiện Dự Đoán và In Ra Kết Quả/Thời Gian

In [ ]:
video_files = glob.glob(os.path.join(INPUT_FOLDER, "drone_video.mp4"), recursive=True)
all_predicted_time = []
all_result = []

print(f"Found {len(video_files)} test cases.")

for video_path in video_files:
    video_id = os.path.basename(os.path.dirname(video_path))
    
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    video_results = []
    
    t1 = time.time() 
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        bbox = model.predict_streaming(frame_rgb, frame_idx)
        
        if bbox is not None:
            video_results.append({
                "frame": frame_idx,
                "x1": bbox[0],
                "y1": bbox[1],
                "x2": bbox[2],
                "y2": bbox[3]
            })
        frame_idx += 1
        
    cap.release()
    
    # Postprocess (Format kết quả)
    result_entry = {
        "video_id": video_id,
        "detections": [{"bboxes": bboxes_per_frame}] if bboxes_per_frame else []
    }
    all_result.append(result_entry)
    
    t2 = time.time() # Kết thúc tính giờ
    
    # Tính thời gian (milliseconds)
    predicted_time = int((t2 - t1) * 1000)
    all_predicted_time.append((video_id, predicted_time))

# 1. Lưu file JSON kết quả (Bắt buộc giống logic predict.py)
with open(OUTPUT_JSON, 'w') as f:
    json.dump(all_result, f)

# 2. Lưu file CSV thời gian
df_time = pd.DataFrame(all_predicted_time, columns=['id', 'time'])
df_time.insert(1, 'answer', 'jupyter_submission.json') 
df_time.to_csv(OUTPUT_CSV, index=False)

print(f"Saved {OUTPUT_JSON}")
print(f"Saved {OUTPUT_CSV}")